In [1]:
import pandas as pd
import numpy as np
import json
import re

import bs4
import urllib3

In [2]:
date = ["2018-02-21", "2018-02-22", "2018-02-23", "2018-02-26", "2018-02-27", "2018-02-28"]

In [3]:
time = []
for i in range(9,16):
    i = str("0") + str(i) if i < 10 else str(i)
    for j in range(0, 60, 10):
        j = str("00") if j == 0 else str(j)
        time.append(i+j)
del time[41]
del time[40]
time[0] = "0901"

In [4]:
def get_market_price(diction, mkt, date, minute):
    
    target = "finance.naver.com/sise/sise_index_time.nhn?code=" + mkt + "&thistime=" + re.sub('[-]', '', date) + minute + "01&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')

    tag = soup.find_all("tr")[2]
    
    index = float(tag.find("td", class_ = "number_1").string.replace(",", ""))

    if "상승" in str(tag):
        direction = 1
    elif "하락" in str(tag):
        direction = -1
    else:
        direction = "error"

    magnitude = tag.find("td", class_ = "rate_down").text.strip("\t\n")
    
    delta = direction * float(magnitude)
    
    diction[date + " " + minute[0:2] + ":" + minute[2:4]] = ((index * 100) / (index - delta)) - 100
    
    return None

In [ ]:
market = "KOSDAQ"
rv = {}
c = 0
for d in date:
    for m in time:
        c += 1
        get_market_price(rv, market, d, m)
        print(c)

In [ ]:
'''
import json
json = json.dumps(rv, ensure_ascii=False)
f = open("KOSDAQ_Feb21_Feb28.json","w")
f.write(json)
f.close()
'''